In [11]:
import pandas as pd
import numpy as np

# 8A

In [12]:
# columns = [ 'NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth', 'Money', 'Other']
nodur = pd.read_csv('NoDur.csv', delimiter=';')
durbl = pd.read_csv('Durbl.csv', delimiter=';')
manuf = pd.read_csv('Manuf.csv', delimiter=';')
enrgy = pd.read_csv('Enrgy.csv', delimiter=';')
chems = pd.read_csv('Chems.csv', delimiter=';')
buseq = pd.read_csv('BusEq.csv', delimiter=';')
telcm = pd.read_csv('Telcm.csv', delimiter=';')
utils = pd.read_csv('Utils.csv', delimiter=';')
shops = pd.read_csv('Shops.csv', delimiter=';')
hlth = pd.read_csv('Hlth.csv', delimiter=';')
money = pd.read_csv('Money.csv', delimiter=';')
other = pd.read_csv('Other.csv', delimiter=';')


In [13]:
industry_dfs = {
    'nodur': nodur,
    'durbl': durbl,
    'manuf': manuf,
    'enrgy': enrgy,
    'chems': chems,
    'buseq': buseq,
    'telcm': telcm,
    'utils': utils,
    'shops': shops,
    'hlth': hlth,
    'money': money,
    'other': other
}


In [14]:
industries = pd.DataFrame(columns=['mean', 'std', 'sharpe', 't-stat'])

# for each industry dataframe compute mean, std, sharpe ratio and t-stat from column 'fund2'
for name, df in industry_dfs.items():
    mean = df['fund2'].mean() * 12
    std = df['fund2'].std() * np.sqrt(12)
    excess = df['fund2'] - df['rf']
    sharpe = (excess.mean() / df['fund2'].std()) * np.sqrt(12)
    t_stat = mean / (std / np.sqrt(len(df)))
    row = pd.DataFrame({'mean': [mean], 'std': [std], 'sharpe': [sharpe], 't-stat': [t_stat]}, index=[name])
    industries = pd.concat([industries, row])


C:\Users\ratas\AppData\Local\Temp\ipykernel_19928\2461961039.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  industries = pd.concat([industries, row])


In [15]:
industries

,mean,std,sharpe,t-stat
nodur,0.116560,0.100687,0.735613,29.400683
durbl,0.086383,0.101056,0.435892,21.726169
manuf,0.133312,0.100945,0.901051,33.592058
enrgy,0.097414,0.100699,0.546024,24.549166
chems,0.099348,0.100905,0.565029,25.024217
buseq,0.110592,0.100499,0.678975,27.990615
telcm,0.082913,0.099868,0.404953,21.068719
utils,0.102144,0.101074,0.589256,25.506080
shops,0.116886,0.100560,0.741374,29.542881
hlth,0.098812,0.100315,0.563013,25.035798


# 8B

In [16]:
# Step 1: Ensure all dataframes have a common column 'date' for merging
for df in industry_dfs.values():
    df.set_index('date', inplace=True)

# Step 2: Concatenate all dataframes along column axis
ew_industry_neutral = pd.concat(industry_dfs.values(), axis=1)

In [17]:

# Step 3: Create a new column 'fund2_avg' that is the average of all 'fund2' columns
ew_industry_neutral['fund2_avg'] = ew_industry_neutral['fund2'].mean(axis=1)
ew_industry_neutral['rf_unique'] = ew_industry_neutral.iloc[:, 0]
# Keep only the 'fund2_avg' column
ew_industry_neutral = ew_industry_neutral[['fund2_avg', 'rf_unique']].dropna()
ew_industry_neutral

,fund2_avg,rf_unique
date,,
1973-01-01,0.017409,0.004643
1973-02-01,0.005442,0.004650
1973-03-01,0.004460,0.004896
1973-04-01,0.008976,0.004989
1973-05-01,0.008716,0.005439
...,...,...
2023-08-01,0.016029,0.004463
2023-09-01,-0.006345,0.004472
2023-10-01,0.008883,0.004475


In [18]:

ew_industry_neutral_mean = ew_industry_neutral['fund2_avg'].mean() * 12
ew_industry_neutral_std = ew_industry_neutral['fund2_avg'].std() * np.sqrt(12)
excess_ew_industry_neutral = ew_industry_neutral['fund2_avg'] - ew_industry_neutral['rf_unique']
ew_industry_neutral_sharpe = excess_ew_industry_neutral.mean() / ew_industry_neutral["fund2_avg"].std() * np.sqrt(12)

# Print the results
print('Equal Weighted industry_neutral:')
print('Mean:', ew_industry_neutral_mean)
print('Standard Deviation:', ew_industry_neutral_std)
print('Sharpe Ratio:', ew_industry_neutral_sharpe)
print("\n")


Equal Weighted industry_neutral:
Mean: 0.11034267343290158
Standard Deviation: 0.05949854203963502
Sharpe Ratio: 1.1403464888043717




# 8C

In [19]:
import statsmodels.api as sm
# Load the datasets
french_factors = pd.read_csv('french_factors.csv', delimiter=';')
industry_returns = pd.read_csv('industry_returns.csv', delimiter=';')
strategy_returns = ew_industry_neutral.reset_index()

# Rename columns for clarity
french_factors.columns = ['date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
industry_returns.columns = ['date', 'NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth', 'Money', 'Other']
strategy_returns.columns = ['date', 'fund2_avg', 'rf_unique']

# Merge data on 'date'
merged_data = pd.merge(strategy_returns, french_factors, on='date')
merged_data = pd.merge(merged_data, industry_returns, on='date')

# Drop rows with missing values
merged_data = merged_data.dropna()

# Extract the dependent variable 
Y = merged_data['fund2_avg'] - merged_data['rf_unique']

# Extract the independent variables (12 industry returns and Fama-French 5 factors)
X = merged_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth', 'Money', 'Other']].astype(float)

# Add a constant term for the intercept
X = sm.add_constant(X)

# Perform the regression
model = sm.OLS(Y, X).fit()

# Display the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     81.88
Date:                Fri, 21 Jun 2024   Prob (F-statistic):          7.91e-224
Time:                        12:13:43   Log-Likelihood:                 7054.8
No. Observations:                2440   AIC:                        -1.407e+04
Df Residuals:                    2422   BIC:                        -1.397e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0038      0.000     10.442      0.0